# Jupyter Sigstore Integration Example (Google Colab)

This notebook demonstrates how to use the jup-sigstore package to sign and verify Hugging Face models within Google Colab.

## Setup

First, install the core dependencies while maintaining Colab compatibility:

In [ ]:
# Install sigstore and other dependencies
%pip install --no-deps sigstore transformers google-auth google-auth-oauthlib nest-asyncio

# Force specific versions for Colab compatibility
%pip install 'ipykernel==5.5.6' 'jupyter-client==6.1.12' 'jupyter-server<2.0.0'

## Install the jup-sigstore package

Install directly from GitHub with --no-deps to prevent dependency conflicts:

In [ ]:
%pip install --no-deps git+https://github.com/yourusername/jup-sigstore.git

## Restart Runtime

After installation, you need to restart the runtime. Click Runtime > Restart runtime, then continue running the cells below.

## Import Dependencies

In [ ]:
from jup_sigstore import ModelSigner
from transformers import AutoTokenizer
from google.colab import auth
import nest_asyncio
import asyncio
import json

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

## Authenticate with Google

This will use Colab's built-in authentication:

In [ ]:
auth.authenticate_user()

## Initialize the Signer

In [ ]:
signer = ModelSigner()

## Load and Sign a Model

We'll use a simple tokenizer as an example:

In [ ]:
async def sign_tokenizer():
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    
    # Sign the tokenizer files
    result = await signer.sign_model(
        tokenizer=tokenizer,
        save_path="./signed_tokenizer"
    )
    
    print("\nSigning completed. Results:")
    print(json.dumps(result, indent=2))
    
    return result

# Run the signing process
result = await sign_tokenizer()

## Verify the Signed Files

In [ ]:
async def verify_signed_files():
    verification = await signer.verify_model("./signed_tokenizer")
    
    print("\nVerification results:")
    for filename, result in verification:
        print(f"\n{filename}:")
        print(f"Verified: {result['success']}")
        if result['success']:
            print(f"Signed by: {result['signer']}")
            print(f"Timestamp: {result['timestamp']}")
        else:
            print(f"Error: {result['error']}")

# Run the verification process
await verify_signed_files()